In [11]:
import socket
import random
import time

In [12]:
def generate_random_move():     # ATTAXX
    x = random.randint(0, 3)
    y = random.randint(0, 3)
    x2 = random.randint(0, 3)
    y2 = random.randint(0, 3)
    return f"MOVE {x},{y},{x2},{y2}"

In [13]:
# def generate_random_move():     # GO
#     x = random.randint(0, 6)
#     y = random.randint(0, 6)
#     return f"MOVE {x},{y}"

In [14]:
def connect_to_server(host='localhost', port=12345):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    response = client_socket.recv(1024).decode()
    print(f"Server ResponseINIT: {response}")
    
    Game = response[-4:]
    print("Playing:", Game)
    
    if "1" in response:
        ag=1
    else:
        ag=2
    first=True
    
    while True:
        # Generate and send a random move
        if ag == 1 or not first:
            move = generate_random_move()
            time.sleep(1)
            client_socket.send(move.encode())
            print("Send:",move)
        
            # Wait for server response
            response = client_socket.recv(1024).decode()
            print(f"Server Response1: {response}")
            if response == "INVALID":
                continue
            if "END" in response: break
         
        first=False
        response = client_socket.recv(1024).decode()
        print(f"Server Response2: {response}")
        if "END" in response: break

        # Add some condition to break the loop, if necessary
        # Example: If server sends a certain message, or after a number of moves

    client_socket.close()

In [15]:
if __name__ == "__main__":
    connect_to_server()

Server ResponseINIT: AG1 G7x7
Playing: G7x7


Send: MOVE 2,0
Server Response1: VALID
Server Response2: MOVE 6,4
Send: MOVE 1,1
Server Response1: VALID
Server Response2: MOVE 1,3
Send: MOVE 1,3
Server Response1: INVALID
Send: MOVE 1,5
Server Response1: VALID
Server Response2: MOVE 6,2
Send: MOVE 2,5
Server Response1: VALID
Server Response2: MOVE 5,1
Send: MOVE 3,5
Server Response1: VALID
Server Response2: MOVE 0,6
Send: MOVE 5,5
Server Response1: VALID
Server Response2: MOVE 6,3
Send: MOVE 0,4
Server Response1: VALID
Server Response2: MOVE 1,4
Send: MOVE 0,4
Server Response1: INVALID
Send: MOVE 1,4
Server Response1: INVALID
Send: MOVE 5,0
Server Response1: VALID
Server Response2: MOVE 3,0
Send: MOVE 6,1
Server Response1: VALID
Server Response2: MOVE 0,2
Send: MOVE 0,6
Server Response1: INVALID
Send: MOVE 5,0
Server Response1: INVALID
Send: MOVE 5,3
Server Response1: VALID
Server Response2: MOVE 4,2
Send: MOVE 5,5
Server Response1: INVALID
Send: MOVE 4,6
Server Response1: VALID
Server Response2: MOVE 0,5
Send: MOVE 6,6
Server Respon

ConnectionResetError: [WinError 10054] Uma ligação existente foi forçada a fechar pelo anfitrião remoto